In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [46]:
from cifar_input import cifar10_input
import Inception_cifar10 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [22]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(50,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.1
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(10):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 2.31482
step 1, cross_entropy 2.30286
step 2, cross_entropy 2.31509
step 3, cross_entropy 2.2932
step 4, cross_entropy 2.29788
step 5, cross_entropy 2.29182
step 6, cross_entropy 2.28482
step 7, cross_entropy 2.30526
step 8, cross_entropy 2.32774
step 9, cross_entropy 2.29562


In [32]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy 2.34179
step 100, cross_entropy 2.20467
step 150, cross_entropy 2.11237
step 200, cross_entropy 2.05938
step 250, cross_entropy 2.01351
test accuracy 0.236
step 300, cross_entropy 2.01619
step 350, cross_entropy 1.96637
step 400, cross_entropy 1.9585
step 450, cross_entropy 1.9671
step 500, cross_entropy 1.93296
test accuracy 0.236
step 550, cross_entropy 1.94518
step 600, cross_entropy 1.89222
step 650, cross_entropy 1.88125
step 700, cross_entropy 1.83801
step 750, cross_entropy 1.85326
test accuracy 0.317
step 800, cross_entropy 1.86021
step 850, cross_entropy 1.83193
step 900, cross_entropy 1.80373
step 950, cross_entropy 1.77622
step 1000, cross_entropy 1.78349
test accuracy 0.332
step 1050, cross_entropy 1.80523
step 1100, cross_entropy 1.79015
step 1150, cross_entropy 1.73958
step 1200, cross_entropy 1.72495
step 1250, cross_entropy 1.76043
test accuracy 0.333
step 1300, cross_entropy 1.75862
step 1350, cross_entropy 1.7238
step 1400, cross_entropy 1.73104

In [48]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.13957
test accuracy 0.181
step 399, cross_entropy 1.934
test accuracy 0.223
step 599, cross_entropy 1.80512
test accuracy 0.238
step 799, cross_entropy 2.28157
test accuracy 0.289
step 999, cross_entropy 2.06207
test accuracy 0.309
step 1199, cross_entropy 2.1219
test accuracy 0.285
step 1399, cross_entropy 1.53047
test accuracy 0.296
step 1599, cross_entropy 1.7484
test accuracy 0.307
step 1799, cross_entropy 2.03164
test accuracy 0.237
step 1999, cross_entropy 1.63983
test accuracy 0.318
step 2199, cross_entropy 1.60477
test accuracy 0.362
step 2399, cross_entropy 1.53424
test accuracy 0.352
step 2599, cross_entropy 2.46255
test accuracy 0.409
step 2799, cross_entropy 1.54163
test accuracy 0.388
step 2999, cross_entropy 1.70525
test accuracy 0.423
step 3199, cross_entropy 1.45204
test accuracy 0.415
step 3399, cross_entropy 1.80031
test accuracy 0.427
step 3599, cross_entropy 1.4119
test accuracy 0.386
step 3799, cross_entropy 1.68201
test accuracy 0.419
ste

In [60]:
with tf.device('/cpu:0'):
    for i in range(2000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.266088
test accuracy 0.815
step 399, cross_entropy 0.131186
test accuracy 0.825
step 599, cross_entropy 0.228554
test accuracy 0.799
step 799, cross_entropy 0.442839
test accuracy 0.79
step 999, cross_entropy 0.463778
test accuracy 0.798
step 1199, cross_entropy 0.426037
test accuracy 0.802
step 1399, cross_entropy 0.465367
test accuracy 0.798
step 1599, cross_entropy 0.153666
test accuracy 0.8
step 1799, cross_entropy 0.368435
test accuracy 0.803
step 1999, cross_entropy 0.501914
test accuracy 0.796
highest accuracy: 0.825


In [59]:
rate=0.0004